In [ ]:
import os
import json
import requests

In [ ]:
#google api url
url = 'https://places.googleapis.com/v1/places:searchText'

In [ ]:
headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': os.environ["GOOGLE_API_KEY"],  # Replace 'API_KEY' with your actual Google Places API key
        'X-Goog-FieldMask': '*'
    }

In [ ]:
request_body = {
    "textQuery": "Recyclying centers",
    "locationBias": {
        "circle": {
            "center": {
                "latitude": 34.0549,
                "longitude": -118.2426
            },
            "radius": 100.0
        }
    }
}

In [ ]:
response = requests.post(url, headers=headers, json=request_body)

In [ ]:
output = response.json()

In [ ]:
for row in output['places']:
    print(row['displayName']['text'])
    print(row['formattedAddress'])

In [ ]:
output

In [ ]:
len(output['places'])

In [ ]:
output['places'][0].keys()

In [ ]:
output['places'][0]['displayName']['text']

In [ ]:
output['places'][0]['location']

In [2]:
import os
from fastmcp import FastMCP

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

test_mcp = FastMCP("Google Places")

@test_mcp.tool(title="Places Locater")
def get_places(query: str, latitude: float, longitude: float) -> dict:
    """Function that leverages the Google Places API to find locations near the latidute and longitude given.

    Args:
        query: The type of location you are searching for (in example "Recycling center")
        latitude: The current latitude of the user
        longitude: The current longitude of the user

    Returns:
        Dictionary with location details, results and metadata
    """
    #the google api url
    url = 'https://places.googleapis.com/v1/places:searchText'
    #headers used for the request
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': GOOGLE_API_KEY,
        'X-Goog-FieldMask': '*'
    }
    #the request body used for the api request
    request_body = {
        "textQuery": query,
        "locationBias": {
            "circle": {
                "center": {
                    "latitude": latitude,
                    "longitude": longitude
                },
                "radius": 100.0
            }
        }
    }
    #get the response
    response = requests.post(url, headers=headers, json=request_body)
    #get the json object
    output = response.json()
    #save the locations in a dictionary
    locations = []
    #loop through the output
    for row in output['places']:
        locations.append({
            "name": row["displayName"]["text"],
            "address": row["formattedAddress"]
        })
    return {
        "query": query,
        "latitude_used": latitude,
        "longitude_used": longitude,
        "results": locations
    }



In [5]:
from langchain_openai import ChatOpenAI

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

llm = ChatOpenAI(model="gpt-4.1-mini")

In [10]:
from fastmcp import Client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_core.messages import HumanMessage

test_client = Client(test_mcp)

async def test_chat():
    async with test_client:
        tools = await load_mcp_tools(test_client.session)
        agent = create_react_agent(model=llm, tools=tools)
        while True:
            user_input = input("User:\t")
            if user_input.lower() in ["quit", "q"]:
                print("Exiting chat.")
                break
            message_template = {"messages": [HumanMessage(content=user_input)]}
            async for message in agent.astream(message_template):
                print(message)


In [11]:
await test_chat()

{'agent': {'messages': [AIMessage(content='Sure! I can help you find recycling centers or places that accept recyclable materials in Los Angeles. Could you please specify what type of item you want to recycle? For example, electronics, plastic, paper, metal, or something else?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 133, 'total_tokens': 181, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6d7dcc9a98', 'id': 'chatcmpl-CGallFSv6OAc9J1H00fpZN9ZOpQU6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bb36d47b-4f7b-454a-ad27-4e7b4035293a-0', usage_metadata={'input_tokens': 133, 'output_tokens': 48, 'total_tokens': 181, 'input_token_details': {'audio': 0, 'cache_rea

[09/16/25 17:57:59] ERROR    Error calling tool 'get_places'                                    ]8;id=84904;file:///Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/python3.12/site-packages/fastmcp/tools/tool_manager.py\tool_manager.py]8;;\:]8;id=640979;file:///Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/python3.12/site-packages/fastmcp/tools/tool_manager.py#233\233]8;;\
                             ╭────────────── Traceback (most recent call last) ───────────────╮                    
                             │ /Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/p │                    
                             │ ython3.12/site-packages/fastmcp/tools/tool_manager.py:224 in   │                    
                             │ call_tool                                                      │                    
                             │                                                                │                    
                             │   221 │   │   │   │   raise NotFoundError(f"Tool {key!r} not f │                    
                             │   222 │   │   │                                                │                    
                             │   223 │   │   │   try:                                         │                    
                             │ ❱ 224 │   │   │   │   return await tool.run(arguments)         │                    
                             │   225 │   │   │                                                │                    
                             │   226 │   │   │   # raise ToolErrors as-is                     │                    
                             │   227 │   │   │   except ToolError as e:                       │                    
                             │                                                                │                    
                             │ /Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/p │                    
                             │ ython3.12/site-packages/fastmcp/tools/tool.py:314 in run       │                    
                             │                                                                │                    
                             │   311 │   │   │   arguments[context_kwarg] = get_context()     │                    
                             │   312 │   │                                                    │                    
                             │   313 │   │   type_adapter = get_cached_typeadapter(self.fn)   │                    
                             │ ❱ 314 │   │   result = type_adapter.validate_python(arguments) │                    
                             │   315 │   │                                                    │                    
                             │   316 │   │   if inspect.isawaitable(result):                  │                    
                             │   317 │   │   │   result = await result                        │                    
                             │                                                                │                    
                             │ /Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/p │                    
                             │ ython3.12/site-packages/pydantic/type_adapter.py:421 in        │                    
                             │ validate_python                                                │                    
                             │                                                                │                    
                             │   418 │   │   │   │   code='validate-by-alias-and-name-false', │                    
                             │   419 │   │   │   )                                            │                    
                             │   420 │   │                                                    │                

{'tools': {'messages': [ToolMessage(content='Error: ToolException("Error calling tool \'get_places\': name \'requests\' is not defined")\n Please fix your mistakes.', name='get_places', id='e9f08234-7de8-46e1-a030-31b37a0a219e', tool_call_id='call_ZLGbo8Jx3h3seDjdbwDYTqmf', status='error')]}}
{'agent': {'messages': [AIMessage(content='It seems there was an error trying to locate recycling centers near you in Los Angeles. Let me try again to find recycling centers nearby.', additional_kwargs={'tool_calls': [{'id': 'call_qhNk4Vtn6uka316EUKTBal7P', 'function': {'arguments': '{"query":"Recycling center","latitude":34.0522,"longitude":-118.2437}', 'name': 'get_places'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 201, 'total_tokens': 260, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0,

[09/16/25 17:58:03] ERROR    Error calling tool 'get_places'                                    ]8;id=731084;file:///Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/python3.12/site-packages/fastmcp/tools/tool_manager.py\tool_manager.py]8;;\:]8;id=100440;file:///Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/python3.12/site-packages/fastmcp/tools/tool_manager.py#233\233]8;;\
                             ╭────────────── Traceback (most recent call last) ───────────────╮                    
                             │ /Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/p │                    
                             │ ython3.12/site-packages/fastmcp/tools/tool_manager.py:224 in   │                    
                             │ call_tool                                                      │                    
                             │                                                                │                    
                             │   221 │   │   │   │   raise NotFoundError(f"Tool {key!r} not f │                    
                             │   222 │   │   │                                                │                    
                             │   223 │   │   │   try:                                         │                    
                             │ ❱ 224 │   │   │   │   return await tool.run(arguments)         │                    
                             │   225 │   │   │                                                │                    
                             │   226 │   │   │   # raise ToolErrors as-is                     │                    
                             │   227 │   │   │   except ToolError as e:                       │                    
                             │                                                                │                    
                             │ /Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/p │                    
                             │ ython3.12/site-packages/fastmcp/tools/tool.py:314 in run       │                    
                             │                                                                │                    
                             │   311 │   │   │   arguments[context_kwarg] = get_context()     │                    
                             │   312 │   │                                                    │                    
                             │   313 │   │   type_adapter = get_cached_typeadapter(self.fn)   │                    
                             │ ❱ 314 │   │   result = type_adapter.validate_python(arguments) │                    
                             │   315 │   │                                                    │                    
                             │   316 │   │   if inspect.isawaitable(result):                  │                    
                             │   317 │   │   │   result = await result                        │                    
                             │                                                                │                    
                             │ /Users/aureliobarrios/GitHub/kura-final-project/kuraVENV/lib/p │                    
                             │ ython3.12/site-packages/pydantic/type_adapter.py:421 in        │                    
                             │ validate_python                                                │                    
                             │                                                                │                    
                             │   418 │   │   │   │   code='validate-by-alias-and-name-false', │                    
                             │   419 │   │   │   )                                            │                    
                             │   420 │   │                                                    │               

{'tools': {'messages': [ToolMessage(content='Error: ToolException("Error calling tool \'get_places\': name \'requests\' is not defined")\n Please fix your mistakes.', name='get_places', id='f97d87fb-4e4e-482a-8f50-e499964aaff7', tool_call_id='call_qhNk4Vtn6uka316EUKTBal7P', status='error')]}}
{'agent': {'messages': [AIMessage(content='I currently have a technical issue with accessing live location data for recycling centers. However, I can still help by suggesting some well-known recycling centers in Los Angeles or guiding you on how to find them via Google Maps or local government resources. Would you like me to provide that information?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 296, 'total_tokens': 353, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'mode